<a href="https://colab.research.google.com/github/jojoconverteo/Evaneos/blob/main/Evaneos_Nettoyage_des_donn%C3%A9es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://www.offremedia.com/sites/default/files/vignette/article/converteo-logo.png)

# **Data cleaning**


**Contexte** : On cherche à savoir si un client mobile est à risque de désengagement


---------------------------------

**Dictionnaire :**

Churn (Cible) :
-  1 if customer cancelled service, 0 if not

AccountWeeks : 
- number of weeks customer has had active account


DataPlan : 
- 1 if customer has data plan, 0 if not

DataUsage : 
 - gigabytes of monthly data usage


CustServCalls : 
- number of calls into customer service


DayMins :
- average daytime minutes per month


DayCalls : 
- average number of daytime calls


MonthlyCharge :
- average monthly bill


OverageFee :
- largest overage fee in last 12 months

ContractRenewal :
- 1 if customer recently renewed contract, 0 if not


RoamMins : 
- average number of roaming minutes [LE ROAMING, C'EST QUOI ?](https://www.sfrbusiness.fr/room/communications-unifiees/roaming-c-est-quoi.html)


In [ ]:
#@title

!pip install plotly --upgrade
!pip install -U pandas_profiling

from pandas_profiling import ProfileReport
from google.colab import drive
import os 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import typing
from typing import List
import numpy as np


import warnings
warnings.filterwarnings('ignore')

drive.mount('/content/drive')
sep = os.sep
str_path_to_file = f"/content/drive/My Drive/Cours Data/Classification/Data/telecom_churn.csv"

     |████████████████████████████████| 13.1MB 285kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 266kB 5.2MB/s 
     |████████████████████████████████| 71kB 5.6MB/s 
     |████████████████████████████████| 604kB 22.2MB/s 
     |████████████████████████████████| 3.1MB 31.6MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 296kB 42.1MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=217841b673f68851842a42668ba9915375195a807bb8c474e938d2685a31ed19
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291991 sha256=c05d19700dad675d636a5bbd45e78b039f88d5c017d6f76fefe779cf43d3ff52
  Stored in direct

In [ ]:
#@title

df_train_data = pd.read_csv(str_path_to_file, encoding='cp424')

UnicodeDecodeError: ignored

In [ ]:
#@title

import chardet

with open(str_path_to_file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

result

{'confidence': 1.0, 'encoding': 'ascii', 'language': ''}

In [ ]:
#@title

df_train_data = pd.read_csv(str_path_to_file, encoding='ascii')

In [ ]:
#@title
def func_create_noise(df_train_data: pd.DataFrame, coloumns_cat_2_category: List) -> pd.DataFrame:
  """
  Fonction qui permet de creer du bruit


  Parameters:
  ----------------------------
    df_train_data: pd.DataFrame 
    Dataframe d'entree 


  Return:
  -----------------------------
    df_train_data_suffle: pd.DataFrame
    Dataframe modifier  

  """
  dict_create_noise_columns_cat = {1 : 'Yes', 0 : 'No'}
  df_train_data_suffle = df_train_data.sample(frac=1)

  for col in columns_cat_2_category: 
    df_train_data_suffle[col] = df_train_data_suffle[col].apply(lambda x: dict_create_noise_columns_cat[x])

  str_col_noise_cat = 'ContractRenewal'
  str_col_noise_num = 'MonthlyCharge'

  df_train_data_suffle.sort_values(by=str_col_noise_cat, ascending=False).reset_index(inplace=True)
  df_train_data_suffle.loc[(df_train_data_suffle.ContractRenewal == 'Yes') & df_train_data_suffle.index.isin(list(range(1, 50))), str_col_noise_cat]  = 'YiS'
  df_train_data_suffle.loc[(df_train_data_suffle.ContractRenewal == 'Yes') & df_train_data_suffle.index.isin(list(range(100, 150))), str_col_noise_cat]  = 'yEs'
  df_train_data_suffle.loc[(df_train_data_suffle.ContractRenewal == 'Yes') & df_train_data_suffle.index.isin(list(range(200, 250))), str_col_noise_cat]  = 'yEssi'
  df_train_data_suffle.loc[df_train_data_suffle.index.isin(list(range(500, 520))), str_col_noise_cat]  = np.NaN

  df_train_data_suffle.sort_values(by=str_col_noise_cat, ascending=True).reset_index(inplace=True)
  df_train_data_suffle.loc[(df_train_data_suffle.ContractRenewal == 'No') & df_train_data_suffle.index.isin(list(range(1,200))), str_col_noise_cat]  = 'NOOO'
  df_train_data_suffle.loc[(df_train_data_suffle.ContractRenewal == 'No') & df_train_data_suffle.index.isin(list(range(200,400))), str_col_noise_cat]  = 'NOpe'

  df_train_data_suffle.loc[df_train_data_suffle.index.isin(list(range(1,10))), str_col_noise_num]  = np.NaN
  return df_train_data_suffle

columns_cat_2_category=['Churn', 'ContractRenewal', 'DataPlan']
df_train_data_suffle = func_create_noise(df_train_data, columns_cat_2_category)

In [ ]:
#@title

df_train_data_suffle

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
2509,No,108,Yes,No,0.00,2,73.8,105,25.0,7.17,10.9
3015,No,132,Yes,Yes,1.92,2,113.2,96,62.2,13.50,7.1
2038,Yes,136,Yes,No,0.45,1,269.8,106,70.5,11.44,10.1
2575,Yes,131,Yes,No,0.00,1,131.6,95,38.0,8.97,15.5
683,No,123,Yes,No,0.00,1,159.1,94,48.0,12.08,6.5
...,...,...,...,...,...,...,...,...,...,...,...
2235,No,117,Yes,Yes,2.62,1,207.6,65,75.2,7.64,9.7
304,No,136,Yes,No,0.34,1,92.0,117,41.4,12.68,10.3
2198,No,127,Yes,No,0.00,0,261.7,105,60.0,9.09,3.3
2332,No,103,Yes,No,0.00,1,158.7,90,44.0,9.92,10.5


### Données manquantes et inconsistantes
------------------------

In [ ]:
#@title

df_train_data_suffle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3333 entries, 2509 to 2361
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Churn            3333 non-null   object 
 1   AccountWeeks     3333 non-null   int64  
 2   ContractRenewal  3313 non-null   object 
 3   DataPlan         3333 non-null   object 
 4   DataUsage        3333 non-null   float64
 5   CustServCalls    3333 non-null   int64  
 6   DayMins          3333 non-null   float64
 7   DayCalls         3333 non-null   int64  
 8   MonthlyCharge    3324 non-null   float64
 9   OverageFee       3333 non-null   float64
 10  RoamMins         3333 non-null   float64
dtypes: float64(5), int64(3), object(3)
memory usage: 312.5+ KB


In [ ]:
#@title

df_train_data_suffle.ContractRenewal.value_counts()

Yes      2861
No        282
yEs        46
yEssi      44
YiS        42
NOpe       21
NOOO       17
Name: ContractRenewal, dtype: int64

In [ ]:
#@title

!pip install fuzzywuzzy

import fuzzywuzzy
from fuzzywuzzy import process

In [ ]:
#@title

df_train_data_suffle.ContractRenewal.unique()

array(['Yes', 'No', 'yEs', 'yEssi', 'YiS', nan, 'NOpe', 'NOOO'],
      dtype=object)

In [ ]:
#@title

df_train_data_suffle['ContractRenewal'] = df_train_data_suffle['ContractRenewal'].str.lower()
array_contract_values = df_train_data_suffle['ContractRenewal'].unique()
list_matches = fuzzywuzzy.process.extract("yes", array_contract_values, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

list_matches

[('yes', 100),
 ('yessi', 75),
 ('yis', 67),
 ('nope', 29),
 ('no', 0),
 (nan, 0),
 ('nooo', 0)]

In [ ]:
#@title

list_word_to_replace = [match[0] for match in list_matches[1:3] if match[1] > 65]
df_train_data_suffle['ContractRenewal'] = df_train_data_suffle['ContractRenewal'].apply(lambda x: 'yes' if x in list_word_to_replace else x)
df_train_data_suffle['ContractRenewal'].value_counts()

yes     2993
no       282
nope      21
nooo      17
Name: ContractRenewal, dtype: int64

In [ ]:
#@title

list_matches = fuzzywuzzy.process.extract("no", array_contract_values, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
list_word_to_replace = [match[0] for match in list_matches[0:3] if match[1] > 65]
df_train_data_suffle['ContractRenewal'] = df_train_data_suffle['ContractRenewal'].apply(lambda x: 'no' if x in list_word_to_replace else x)
df_train_data_suffle['ContractRenewal'].value_counts()

yes    2993
no      320
Name: ContractRenewal, dtype: int64

In [ ]:
#@title

df_train_data_suffle['ContractRenewal'].value_counts(dropna=False)

yes    2993
no      320
NaN      20
Name: ContractRenewal, dtype: int64

In [ ]:
#@title

df_train_data_suffle['ContractRenewal'] = df_train_data_suffle['ContractRenewal'].fillna(df_train_data_suffle['ContractRenewal'].mode()[0])
df_train_data_suffle['ContractRenewal'].value_counts(dropna=False)

yes    3013
no      320
Name: ContractRenewal, dtype: int64